In [50]:
import urllib.request
import os.path
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError 

In [158]:
DB_PATH = "data/"
DB_FILE = "ds-plus-final.db"
DB_URL = "https://code.s3.yandex.net/data-scientist/ds-plus-final.db"
DB_TABLE_NAMES = ['contract', 'personal', 'internet', 'phone']
DB_TABLE_DICT = {
    "contract": [
        "customerID",
        "BeginDate",
        "EndDate",
        "Type",
        "PaperlessBilling",
        "PaymentMethod",
        "MonthlyCharges",
        "TotalCharges"
    ],
    "personal": [
        "customerID",
        "gender",
        "SeniorCitizen",
        "Partner",
        "Dependents"
    ],
    "internet": [
        "customerID",
        "InternetService",
        "OnlineSecurity",
        "OnlineBackup",
        "DeviceProtection",
        "TechSupport",
        "StreamingTV",
        "StreamingMovies"
    ],
    "phone": [
        "customerID",
        "MultipleLines"
    ]
}                  
RANDOM_STATE = 110825
TARGET_METRIC = .85
TARGET = "EndDate"

['contract', 'personal', 'internet', 'phone']

In [54]:
if not os.path.exists(DB_PATH+DB_FILE):
    try:
        urllib.request.urlretrieve(DB_URL, DB_PATH+DB_FILE)
        print(f"База данных {DB_FILE} загружена")
    except Exception as err:
        print(f"Ошибка при попытке загрузки базы данных {err}")
else:
    print(f"База данных {DB_FILE} уже загружена")

База данных ds-plus-final.db уже загружена


In [56]:
engine = create_engine(f'sqlite:///{DB_PATH+DB_FILE}', echo=False) 

In [318]:
class TableStructureValidator:
    def __init__(self, engine, tables_dict=None):
        if not tables_dict:
            raise ValueError('Словарь tables_dict должен быть указан и не может быть пустым')
        self.engine = engine
        self.tables_dict = tables_dict
        self.results = {}
        self._validate_tables()
    
    def _get_table_info(self, table_name):
        try:
            query = text(f"PRAGMA table_info({table_name})")
            return pd.read_sql_query(query, con=self.engine)
        except SQLAlchemyError as e:
            print(f"Ошибка при получении информации о таблице {table_name}: {e}")
            return None
            
    def _check_null_values(self, table_name):
        null_counts = {}
        try:
            for column in self.tables_dict[table_name]:
                query = text(f"SELECT COUNT(*) FROM {table_name} WHERE {column} IS NULL")
                null_count = pd.read_sql_query(query, self.engine).iloc[0, 0]
                if null_count > 0:
                    null_counts[column] = null_count
        except Exception as e:
            print(f"Ошибка при проверке NULL-значений в {table_name}.{column}: {e}")
        return null_counts
        
    def _validate_tables(self):        
        for table_name, expected_columns in self.tables_dict.items():
            actual_info = self._get_table_info(table_name)
            actual_columns = actual_info['name'].tolist()
            missing_in_db = set(expected_columns) - set(actual_columns)
            extra_in_db = set(actual_columns) - set(expected_columns)
            null_values = self._check_null_values(table_name)
            self.results[table_name] = {
                'status': 'ok' if not missing_in_db and not extra_in_db else 'mismatch',
                'actual_columns': actual_columns,
                'missing_columns': list(missing_in_db),
                'extra_columns': list(extra_in_db),
                'column_types': dict(zip(actual_info['name'], actual_info['type'])),
                'null_values': null_values,
                'null_values_count': sum(null_values.values())
            }

    def print_info(self):
        for table_name, data in self.results.items():
            print(f"\nТаблица: {table_name}")
            print("-" * 50)
            
            if data['status'] == 'ok':
                print("✅ Структура полностью соответствует ожидаемой")
            else:
                if data['missing_columns']:
                    print(f"❌ Отсутствующие столбцы: {', '.join(data['missing_columns'])}")
                if data['extra_columns']:
                    print(f"❌ Лишние столбцы в БД: {', '.join(data['extra_columns'])}")
                    
            if data['null_values']:
                print("\nПропущенные значения (NULL):")
                for col, count in data['null_values'].items():
                    print(f"  - {col}: {count} пропусков")
                    
            print("\nПроверка столбцов и типов:")
            for col in data['actual_columns']:
                status = "🔴" if col in data['missing_columns'] or col in data['extra_columns'] else "🟢"
                col_type = data['column_types'].get(col)
                print(f"{status} {col}: {col_type}")
            
            print("-" * 50)

In [320]:
# инициализация и запуск валидаци таблиц БД
validator = TableStructureValidator(engine=engine, tables_dict=DB_TABLE_DICT)
validator.print_info()


Таблица: contract
--------------------------------------------------
✅ Структура полностью соответствует ожидаемой

Проверка столбцов и типов:
🟢 customerID: TEXT
🟢 BeginDate: TEXT
🟢 EndDate: TEXT
🟢 Type: TEXT
🟢 PaperlessBilling: TEXT
🟢 PaymentMethod: TEXT
🟢 MonthlyCharges: TEXT
🟢 TotalCharges: TEXT
--------------------------------------------------

Таблица: personal
--------------------------------------------------
✅ Структура полностью соответствует ожидаемой

Проверка столбцов и типов:
🟢 customerID: TEXT
🟢 gender: TEXT
🟢 SeniorCitizen: TEXT
🟢 Partner: TEXT
🟢 Dependents: TEXT
--------------------------------------------------

Таблица: internet
--------------------------------------------------
✅ Структура полностью соответствует ожидаемой

Проверка столбцов и типов:
🟢 customerID: TEXT
🟢 InternetService: TEXT
🟢 OnlineSecurity: TEXT
🟢 OnlineBackup: TEXT
🟢 DeviceProtection: TEXT
🟢 TechSupport: TEXT
🟢 StreamingTV: TEXT
🟢 StreamingMovies: TEXT
------------------------------------------

In [178]:
# # функция для получения структуры таблиц из БД
# def get_table_info(table_name, engine=engine):
#     query = text(f"PRAGMA table_info({table_name})")
#     print(f'Столбцы таблицы {table_name}:')
#     try:
#         print(pd.read_sql_query(query, con=engine)['name'], '\n')
#     except SQLAlchemyError as e:
#         print(f"Ошибка при выполнении запроса: {e}")

def get_table_info(table_name, engine=engine):
    query = text(f"PRAGMA table_info({table_name})")
    print(f'\nСтруктура таблицы {table_name}:')
    print('-' * 40)
    try:
        table_info = pd.read_sql_query(query, con=engine)
        for _, row in table_info.iterrows():
            print(f"{row['name']}: {row['type']} {'(PK)' if row['pk'] else 'not PK'}")
        print('-' * 40, '\n')
    except SQLAlchemyError as e:
        print(f"Ошибка при выполнении запроса: {e}")

In [180]:
# функция для получения количества записей в каждой таблице
def get_table_size(table_name, engine=engine):
    query = text(f'SELECT COUNT(*) FROM {table_name}')
    try:
        print(f'Строк в таблице {table_name}: '
              f'{pd.read_sql_query(query, con=engine).iloc[0,0]}', '\n', '-'*50)
    except SQLAlchemyError as e:
        print(f"Ошибка при выполнении запроса: {e}")

In [182]:
for table in list(DB_TABLE_DICT.keys()):
    get_table_info(table)
    get_table_size(table)


Структура таблицы contract:
----------------------------------------
customerID: TEXT not PK
BeginDate: TEXT not PK
EndDate: TEXT not PK
Type: TEXT not PK
PaperlessBilling: TEXT not PK
PaymentMethod: TEXT not PK
MonthlyCharges: TEXT not PK
TotalCharges: TEXT not PK
---------------------------------------- 

Строк в таблице contract: 7043 
 --------------------------------------------------

Структура таблицы personal:
----------------------------------------
customerID: TEXT not PK
gender: TEXT not PK
SeniorCitizen: TEXT not PK
Partner: TEXT not PK
Dependents: TEXT not PK
---------------------------------------- 

Строк в таблице personal: 7043 
 --------------------------------------------------

Структура таблицы internet:
----------------------------------------
customerID: TEXT not PK
InternetService: TEXT not PK
OnlineSecurity: TEXT not PK
OnlineBackup: TEXT not PK
DeviceProtection: TEXT not PK
TechSupport: TEXT not PK
StreamingTV: TEXT not PK
StreamingMovies: TEXT not PK
-------

In [130]:
# функция для формирования статистик признаков и распределения по категориям
def external_info(df, threshold=10, density_threshold=.01):
    numeric_stats = ['max', 'min', 'median', 'mean']
    stats = []
    for feature in df.columns:
        feature_series = df[feature]
        nunique = feature_series.nunique()
        density = round(nunique / len(df), 2)
        is_numeric = pd.api.types.is_numeric_dtype(feature_series)
        is_string = pd.api.types.is_string_dtype(df[feature])
                                                   
        feature_stats = {
            'feature': feature,
            'dtype': feature_series.dtype,
            'Cardinality': nunique,
            'Card. density': density,
            'Missing': feature_series.isna().sum(),
            'Missing %': feature_series.isna().mean(),
            'Mode': feature_series.mode()[0]
        }
        if is_numeric:
            if density < density_threshold or nunique < threshold:
                feature_stats['Type'] = 'discrete'
            else:
                feature_stats['Type'] = 'continuous'
            for stat in numeric_stats:
                try:
                    feature_stats[stat.capitalize()] = getattr(df[feature], stat)()
                except (AttributeError, TypeError):
                    feature_stats[stat.capitalize()] = 'Не найден'
        else:
            if is_string:
                feature_stats['Type'] = 'categorical'         
            else:
                feature_stats['Type'] = 'unknown'
            for stat in numeric_stats:
                feature_stats[stat.capitalize()] = "-"      
        stats.append(feature_stats)
    return pd.DataFrame(stats)

In [140]:
query = text(f"SELECT * FROM {DB_TABLE_NAMES[0]}")
pd.read_sql_query(query, con = engine)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,31.04
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,2071.84
2,3668-QPYBK,2019-10-01,No,Month-to-month,Yes,Mailed check,53.85,226.17
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.3,1960.6
4,9237-HQITU,2019-09-01,No,Month-to-month,Yes,Electronic check,70.7,353.5
...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.8,2035.2
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.2,7430.4
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.6,325.6
7041,8361-LTMKD,2019-07-01,No,Month-to-month,Yes,Mailed check,74.4,520.8


In [142]:
external_info(pd.read_sql_query(query, con = engine))

,feature,dtype,Cardinality,Card. density,Missing,Missing %,Mode,Type,Max,Min,Median,Mean
0,customerID,object,7043,1.00,0,0.0,0002-ORFBO,categorical,-,-,-,-
1,BeginDate,object,77,0.01,0,0.0,2014-02-01,categorical,-,-,-,-
2,EndDate,object,67,0.01,0,0.0,No,categorical,-,-,-,-
3,Type,object,3,0.00,0,0.0,Month-to-month,categorical,-,-,-,-
4,PaperlessBilling,object,2,0.00,0,0.0,Yes,categorical,-,-,-,-
5,PaymentMethod,object,4,0.00,0,0.0,Electronic check,categorical,-,-,-,-
6,MonthlyCharges,object,1585,0.23,0,0.0,20.05,categorical,-,-,-,-
7,TotalCharges,object,6658,0.95,0,0.0,,categorical,-,-,-,-
